In [1]:
import pymysql 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from collections import defaultdict, Counter
import _pickle as pickle
import random
from scipy.stats import epps_singleton_2samp, wasserstein_distance, ks_2samp
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
from lifelines import KaplanMeierFitter, CoxPHFitter

In [2]:
visit_probability = {}
for visit_id, prob in np.array(pd.read_csv('data/rfr_model_depth_69_trees_190_preds.csv', header=None)):
    visit_probability[int(visit_id)] = prob
    
len(visit_probability)

1573113

In [3]:
# start tunnel first: ssh -f [uni]@mimir.dbmi.columbia.edu -L 3307:127.0.0.1:3306 -N

conn = pymysql.connect(host="127.0.0.1", 
                       user="", #uni
                       port = ,
                       passwd="", #sql password
                       db = "" ) #database
cur = conn.cursor()

In [4]:
phenotype_visits = []
cur.execute('''select phecode, a.visit_id, a.contact_date, b.ed_dt
                from user_vr2430.vfinal_1_predict_covid_conditions a
                join (select visit_id, pat_mrn_id, ed_dt from user_vr2430.vfinal_1_predict_covid_visits) b using (pat_mrn_id)
                join clinical_merge_v5_2022q1.phecode_icd10 using (icd10)
                where a.contact_date >= b.ed_dt and a.visit_id != b.visit_id;''')

for phe, visit, ct, ed_dt in cur.fetchall():
    if visit in visit_probability:
        phenotype_visits.append([phe,visit, ct, ed_dt])

phenotype_visits = np.array(pd.DataFrame(phenotype_visits))

len(phenotype_visits), len(set(phenotype_visits[:,0]))

(7825382, 1048)

In [5]:
cur.execute('''select distinct a.visit_id, a.contact_date, b.ed_dt
                from user_vr2430.vfinal_1_predict_covid_conditions a
                join (select visit_id, pat_mrn_id, ed_dt from user_vr2430.vfinal_1_predict_covid_visits) b using (pat_mrn_id)
                where a.contact_date >= b.ed_dt and a.visit_id != b.visit_id;''')

followup_visits = []
for visit, ct, ed_dt in cur.fetchall():
    if visit in visit_probability:
        followup_visits.append([visit, ct, ed_dt])
        
len(followup_visits)

5002965

In [6]:
followup_visits = np.array(pd.DataFrame(followup_visits))

In [7]:
len(set(phenotype_visits[:,1])), len(set(followup_visits[:,0]))

(277994, 529295)

In [8]:
phenotype_visits_7_all = defaultdict(list)
phenotype_visits_14_all = defaultdict(list)
phenotype_visits_21_all = defaultdict(list)
phenotype_visits_28_all = defaultdict(list)
phenotype_visits_3m_all = defaultdict(list)
phenotype_visits_6m_all = defaultdict(list)
phenotype_visits_9m_all = defaultdict(list)
phenotype_visits_1y_all = defaultdict(list)

for phe, visit_id, diag_date, ed_date in tqdm(phenotype_visits):
        if visit_id not in visit_probability:
            continue
        if (diag_date-ed_date).days <= 7:
            phenotype_visits_7_all[phe].append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 14:
            phenotype_visits_14_all[phe].append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 21:
            phenotype_visits_21_all[phe].append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 28:
            phenotype_visits_28_all[phe].append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 91:
            phenotype_visits_3m_all[phe].append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 183:
            phenotype_visits_6m_all[phe].append([visit_id, diag_date, ed_date])  
        if (diag_date-ed_date).days <= 274:
            phenotype_visits_9m_all[phe].append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 365:
            phenotype_visits_1y_all[phe].append([visit_id, diag_date, ed_date])

            
len(phenotype_visits_7_all), len(phenotype_visits_14_all), len(phenotype_visits_21_all), len(phenotype_visits_28_all), len(phenotype_visits_3m_all) ,len(phenotype_visits_6m_all), len(phenotype_visits_9m_all) ,len(phenotype_visits_1y_all)

100%|██████████| 7825382/7825382 [00:29<00:00, 268994.37it/s]


(984, 1002, 1013, 1018, 1037, 1043, 1048, 1048)

In [10]:
followup_visits_7 = []
followup_visits_14 = []
followup_visits_21 = []
followup_visits_28 = []
followup_visits_3m = []
followup_visits_6m = []
followup_visits_9m = []
followup_visits_1y = []

for visit_id, diag_date, ed_date in tqdm(followup_visits):
        if visit_id not in visit_probability:
            continue
        if (diag_date-ed_date).days <= 7:
            followup_visits_7.append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 14:
            followup_visits_14.append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 21:
            followup_visits_21.append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 28:
            followup_visits_28.append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 91:
            followup_visits_3m.append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 183:
            followup_visits_6m.append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <= 274:
            followup_visits_9m.append([visit_id, diag_date, ed_date])
        if (diag_date-ed_date).days <=365:
            followup_visits_1y.append([visit_id, diag_date, ed_date])

len(followup_visits_7), len(followup_visits_14), len(followup_visits_21), len(followup_visits_28), len(followup_visits_3m) ,len(followup_visits_6m), len(followup_visits_9m) ,len(followup_visits_1y)

100%|██████████| 5002965/5002965 [00:17<00:00, 281157.33it/s]


(266074, 502360, 717314, 922617, 2239595, 3471263, 4184347, 4589280)

In [44]:
followup_visits_7 = np.array(pd.DataFrame(followup_visits_7))
followup_visits_14 = np.array(pd.DataFrame(followup_visits_14))
followup_visits_21 = np.array(pd.DataFrame(followup_visits_21))
followup_visits_28 = np.array(pd.DataFrame(followup_visits_28))
followup_visits_3m = np.array(pd.DataFrame(followup_visits_3m))
followup_visits_6m = np.array(pd.DataFrame(followup_visits_6m))
followup_visits_9m = np.array(pd.DataFrame(followup_visits_9m))
followup_visits_1y = np.array(pd.DataFrame(followup_visits_1y))




In [12]:
followup_tm_7 = defaultdict(list)
followup_tm_14 = defaultdict(list)
followup_tm_21 = defaultdict(list)
followup_tm_28 = defaultdict(list)
followup_tm_3m = defaultdict(list)
followup_tm_6m = defaultdict(list)
followup_tm_9m = defaultdict(list)
followup_tm_1y = defaultdict(list)

for visit_id, ed, st in tqdm(followup_visits_7):
    followup_tm_7[visit_id].append((ed-st).days)
for visit_id, ed, st in tqdm(followup_visits_14):
    followup_tm_14[visit_id].append((ed-st).days)
for visit_id, ed, st in tqdm(followup_visits_21):
    followup_tm_21[visit_id].append((ed-st).days)
for visit_id, ed, st in tqdm(followup_visits_28):
    followup_tm_28[visit_id].append((ed-st).days)
for visit_id, ed, st in tqdm(followup_visits_3m):
    followup_tm_3m[visit_id].append((ed-st).days)
for visit_id, ed, st in tqdm(followup_visits_6m):
    followup_tm_6m[visit_id].append((ed-st).days)
for visit_id, ed, st in tqdm(followup_visits_9m):
    followup_tm_9m[visit_id].append((ed-st).days)    
for visit_id, ed, st in tqdm(followup_visits_1y):
    followup_tm_1y[visit_id].append((ed-st).days)

100%|██████████| 4589280/4589280 [00:02<00:00, 1963396.35it/s]


In [45]:
pickle.dump(phenotype_visits_7_all, open('data/phenotype_visits_7_all.p', 'wb'))
pickle.dump(phenotype_visits_14_all, open('data/phenotype_visits_14_all.p', 'wb'))
pickle.dump(phenotype_visits_21_all, open('data/phenotype_visits_21_all.p', 'wb'))
pickle.dump(phenotype_visits_28_all, open('data/phenotype_visits_28_all.p', 'wb'))
pickle.dump(phenotype_visits_3m_all, open('data/phenotype_visits_3m_all.p', 'wb'))
pickle.dump(phenotype_visits_6m_all, open('data/phenotype_visits_6m_all.p', 'wb'))
pickle.dump(phenotype_visits_9m_all, open('data/phenotype_visits_9m_all.p', 'wb'))
pickle.dump(phenotype_visits_1y_all, open('data/phenotype_visits_1y_all.p', 'wb'))
pickle.dump(followup_visits_7, open('data/followup_visits_7.p', 'wb'))
pickle.dump(followup_visits_14, open('data/followup_visits_14.p', 'wb'))
pickle.dump(followup_visits_21, open('data/followup_visits_21.p', 'wb'))
pickle.dump(followup_visits_28, open('data/followup_visits_28.p', 'wb'))
pickle.dump(followup_visits_3m, open('data/followup_visits_3m.p', 'wb'))
pickle.dump(followup_visits_6m, open('data/followup_visits_6m.p', 'wb'))
pickle.dump(followup_visits_9m, open('data/followup_visits_9m.p', 'wb'))
pickle.dump(followup_visits_1y, open('data/followup_visits_1y.p', 'wb'))
pickle.dump(followup_tm_7, open('data/followup_tm_7.p', 'wb'))
pickle.dump(followup_tm_14, open('data/followup_tm_14.p', 'wb'))
pickle.dump(followup_tm_21, open('data/followup_tm_21.p', 'wb'))
pickle.dump(followup_tm_28, open('data/followup_tm_28.p', 'wb'))
pickle.dump(followup_tm_3m, open('data/followup_tm_3m.p', 'wb'))
pickle.dump(followup_tm_6m, open('data/followup_tm_6m.p', 'wb'))
pickle.dump(followup_tm_9m, open('data/followup_tm_9m.p', 'wb'))
pickle.dump(followup_tm_1y, open('data/followup_tm_1y.p', 'wb'))

In [48]:
survival_results_1y = []
for phe in tqdm([i for i in phenotype_visits_1y_all.keys() if i !='']):
    case_visits_1y = []
    for visit_id, diag_date, ed_date in phenotype_visits_1y_all[phe]:
        if visit_id not in visit_probability:
            continue
        if (diag_date-ed_date).days <= 365:
            case_visits_1y.append([visit_id, (diag_date-ed_date).days])
            
    case_visits_1y = np.array(pd.DataFrame(case_visits_1y))
    non_case_visits_1y = list((set(followup_visits_1y[:,0])-set(case_visits_1y[:,0])))
    
    coxph_model_data = []


    for visit_id in set(case_visits_1y[:,0]):
        time_to_diag = min(case_visits_1y[:,1][case_visits_1y[:,0]==visit_id])
        covid_prob = visit_probability[visit_id]
        coxph_model_data.append([covid_prob, 1, time_to_diag])
    
    for visit_id in non_case_visits_1y:
        time_to_diag = min([365] + followup_tm_1y[visit_id])
        covid_prob = visit_probability[visit_id]
        coxph_model_data.append([covid_prob, 0, time_to_diag])
    
    
    coxph_model_data = pd.DataFrame(coxph_model_data, columns=['covid_prob', 'phenotype', 'days'])
    cph = CoxPHFitter()
    cph.fit(coxph_model_data, 'days', 'phenotype')
    hz = cph.summary['exp(coef)'][0]
    lw = cph.summary['exp(coef) lower 95%'][0]
    up = cph.summary['exp(coef) upper 95%'][0]
    z = cph.summary['z'][0]
    p = cph.summary['-log2(p)'][0]
    survival_results_1y.append([phe, len(case_visits_1y), len(non_case_visits_1y), hz, lw, up, z, p])
    
pd.DataFrame(survival_results_1y, columns=['phe', 'case_n', 'noncase_n', 'hazards ratio', 
                                          '95% confidence interval lower','95% confidence interval upper', 'z','-log2(p)']).to_csv('data/phenotype_copy_new_1y.csv', index=False)

100%|██████████| 1047/1047 [2:45:27<00:00,  9.48s/it] 
